## Neighborhoods in Toronto


### Question 1

#### Import Required Libraries

In [7]:
import pandas as pd
import requests

#### Scrape and Clean Data from the Wikipedia page 

In [8]:
nbrDataAddr = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
nbrData = pd.read_html(nbrDataAddr.text)
nbrData = nbrData[0]

nbrDF = nbrData[nbrData["Borough"] != "Not assigned"]
nbrDF

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Get Shape of the DataFrame

In [9]:
nbrDF.shape

(103, 3)

### Question 2

In [12]:
pip install geocoder

     |████████████████████████████████| 98 kB 6.3 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [13]:
import geocoder

geocodeData = pd.read_csv("http://cocl.us/Geospatial_data")
geocodeData

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
